## Financial Data Analysis
Climate data is from the world bank; read more about these time series [here](https://climateknowledgeportal.worldbank.org/country/uruguay/climate-data-historical).

In [ ]:
!pip install statsmodels --upgrade
!pip install yfinance

In [ ]:
"""
No need to change this block - loads dataset and necessary packages
"""
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import io
import yfinance as yf
url="https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv"
s = requests.get(url).content
companies = pd.read_csv(io.StringIO(s.decode('utf-8')))
print("All available NASDAQ listings: \n", companies.head())
msft = yf.download("MSFT",start='2009-06-01',progress=False)
print("Sample of MSFT stock series: \n", msft.head())
price = msft.reset_index()['Open'].values.reshape(-1,1)
date = msft.reset_index().Date.dt.date.values.reshape(-1,1)
plt.plot(date, price)
plt.title("MSFT stock price over time")
plt.show()
log_price = np.log(price)
plt.plot(date, log_price)
plt.title("log of MSFT stock price over time")
plt.show()

### Transform Data to Stationary
Here, transform the time series to a stationary series. 

Recall your tools from the last couple weeks (you may find it helpful to refer to the solutions from last week [here](https://colab.research.google.com/drive/1vJmoFJu0rSKdLZOqinD6Czr1vrVaEAIl?usp=sharing). 
Recall that a stationary time series: 
1. Has no trend
2. Has no seasonal component
3. Has constant variance 


At the end of this investigation, you should have a stationary variable `stationary_price` that is a stationary transformation of your price data. 

In [ ]:
## Here, add code to apply the necessary transformations. 
# Include any plots that help support your claim. 
# Note these suggested packages
from sklearn import linear_model
<TODO: TRANSFORM DATA> 

In [ ]:
## Some starter code to consider a higher order polynomial
index = msft.reset_index().index.values.reshape(-1,1)
new_x = np.hstack((index, index **2))
<TODO: TRANFORM DATA TO STATIONARY>

In [ ]:
import statsmodels as sm
sm.api.graphics.tsa.plot_acf(<ADD DATA HERE>, lags=30)
plt.show()
sm.api.graphics.tsa.plot_pacf(<ADD DATA HERE>, lags=30)
plt.show();

In [ ]:
# consider adding sinusoidal fit to the model 
from scipy import optimize
period_range = np.arange(1, 100)
mse_list = []

def find_sine_approximation(period, x_data=date, y_data=log_price):
  """
  Finds a best-fitting sinusoidal approximation, for the given frequency. 
  """
  def sine_function(X, amp, phase_shift, mean):
    return (amp * np.sin(1/period * 2 * np.pi * (X - phase_shift)) + mean)
  params, _ = sine_curve_fit = optimize.curve_fit(
    f = sine_function,
    xdata = x_data.flatten(),
    ydata = y_data.flatten(),
    p0 = np.array([3, 1, 0]))
  amp, phase_shift, mean = params
  sin_prediction = sine_function(x_data, amp, phase_shift, mean)
  return sin_prediction
for period in period_range:
  sin_prediction = find_sine_approximation(period, <ADD INDEX HERE> ,
                                           <ADD ENDOGENOUS VARIABLE HERE HERE>)
  mse = np.nanmean((<ADD ENDOGENOUS VARIABLE HERE> - sin_prediction) **2)
  mse_list.append(mse)
plt.plot(period_range, mse_list);
plt.show()
period_guess = period_range[np.argmin(mse_list)]
print("minimizing period is:", period_guess)

In [ ]:
stationary_price = <ADD YOUR BEST GUESS AT THE TRANSFORMATION HERE> 

In [ ]:
# No need to change this block
# We'll be evaluating how well various models forecast - to do so, we split the data
# into train/test sets using the first 80% of the observations as training data
train_test_split = int(len(price) * 0.8)
train_price, test_price = stationary_price[:train_test_split], stationary_price[train_test_split:]
train_date, test_date = date[:train_test_split], date[train_test_split:]
assert(len(train_date) + len(test_date) == len(date))

In [ ]:
# No need to change this block - defines AIC/BIC
from scipy.stats import norm 
def evaluate_AIC(k, residuals):
  """
  Finds the AIC given the number of parameters estimated and 
  the residuals of the model. Assumes residuals are distributed 
  Gaussian with unknown variance. 
  """
  standard_deviation = np.std(residuals)
  log_likelihood = norm.logpdf(residuals, np.mean(residuals), scale=standard_deviation)
  return 2 * k - 2 * np.sum(log_likelihood)
def evaluate_BIC(k, residuals):
  """
  Finds the AIC given the number of parameters estimated and 
  the residuals of the model. Assumes residuals are distributed 
  Gaussian with unknown variance. 
  """
  standard_deviation = np.std(residuals)
  log_likelihood = norm.logpdf(residuals, np.mean(residuals), scale=standard_deviation)
  return k * np.log(len(residuals)) - 2 * np.sum(log_likelihood)

### Find ACF/PACT of Stock Price Data
Plot the ACF and PACF. Use the package referenced [here (ACF)](https://www.statsmodels.org/dev/generated/statsmodels.graphics.tsaplots.plot_acf.html) and [here (PACF)](https://www.statsmodels.org/dev/generated/statsmodels.graphics.tsaplots.plot_pacf.html).

What does this suggest about the AR/MA order of the model? 

In [ ]:
import statsmodels as sm
sm.api.graphics.tsa.plot_acf(train_price, lags=30)
plt.show()
sm.api.graphics.tsa.plot_pacf(train_price, lags=30)
plt.show()

### Fitting AR models
Fit an AR(1) model to the data. Plot the model fit, residuals, and find the AIC and BIC. 

In [ ]:
import statsmodels as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

ar_1 = SARIMAX(train_price, order=<ADD ORDER HERE>).fit()
print(ar_1.summary())
ar_1_predictions = ar_1.predict()
ar_1_residuals = train_price - ar_1_predictions.reshape(-1,1)
ar_1_residuals = ar_1_residuals[1:] # Fitting AR 1 model means removing one observation
plt.plot(train_date, train_price, label='original data')
plt.plot(train_date[1:], ar_1_predictions[1:], 'r', label='fitted line')
plt.show()
plt.plot(train_date[1:], ar_1_residuals, 'o')
plt.show()
print("MSE with AR(1) model:", np.mean(ar_1_residuals**2))
print("AIC with AR(1) model:", evaluate_AIC(<ADD DOF HERE>, ar_1_residuals))
print("BIC with AR(1) model:", evaluate_BIC(<ADD DOF HERE>s, ar_1_residuals))

### Higher order AR models
Fit higher-order AR models to the data. Do these improve the fit? 
Based on the ACF/PACF models, would you expect them to? 

In [ ]:
import statsmodels as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
AR_order = 5
ar_higher = SARIMAX(train_price, order=<ADD ORDER HERE>).fit()
print(ar_higher.summary())
ar_higher_predictions = ar_higher.predict()
ar_higher_residuals = train_price - ar_higher_predictions.reshape(-1,1)
ar_higher_residuals = ar_higher_residuals[1:] # Fitting AR 1 model means removing one observation
plt.plot(train_date, train_price, label='original data')
plt.plot(train_date[1:], ar_higher_predictions[1:], 'r', label='fitted line')
plt.show()
plt.plot(train_date[1:], ar_higher_residuals, 'o')
plt.show()
print("MSE with AR(1) model:", np.mean(ar_higher_residuals**2))
print("AIC with AR(1) model:", evaluate_AIC(<ADD DOF HERE>, ar_higher_residuals))
print("BIC with AR(1) model:", evaluate_BIC(<ADD DOF HERE>, ar_higher_residuals))

In [ ]:
ar_higher.aic

### ARMA Models
Varying the AR and MA parameters, find an optimal fit to the dataset. 
Is it what you expect from the ACF/PACF plots? 

In [ ]:
import statsmodels as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
# Note: it may help compare models to iterate over AIC/BIC. 
# To do so, you can use arma.aic or arma.bic to use the statsmodels computed values

arma = SARIMAX(train_price, order=<ADD ORDER HERE>).fit()
print(arma.summary())
arma_predictions = arma.predict()
arma_residuals = train_price - arma_predictions.reshape(-1,1)
arma_residuals = arma_residuals[1:] # Fitting AR 1 model means removing one observation
plt.plot(train_date, train_price, label='original data')
plt.plot(train_date[1:], arma_predictions[1:], 'r', label='fitted line')
plt.show()
plt.plot(train_date[1:], arma_residuals, 'o')
plt.show()
print("MSE with selected model:", np.mean(arma_residuals**2))
print("AIC with selected model:", evaluate_AIC(<ADD DOF HERE>, arma_residuals))
print("BIC with selected model:", evaluate_BIC(<ADD DOF HERE>, arma_residuals))

In [ ]:
arma = SARIMAX(train_price, order=min_aic_index).fit()
print(arma.summary())
arma_predictions = arma.predict()
arma_residuals = train_price - arma_predictions.reshape(-1,1)
arma_residuals = arma_residuals[1:] # Fitting AR 1 model means removing one observation
plt.plot(train_date, train_price, label='original data')
plt.plot(train_date[1:], arma_predictions[1:], 'r', label='fitted line')
plt.show()
plt.plot(train_date[1:], arma_residuals, 'o')
plt.show()
print("MSE with selected model:", np.mean(arma_residuals**2))
print("AIC with selected model:", evaluate_AIC(sum(min_aic_index)+ 1, arma_residuals))
print("BIC with selected model:", evaluate_BIC(sum(min_bic_index) + 1, arma_residuals))

### Forecasting
Use your preferred model to forecast the remaining time series. 
Plot this on the same axes as the true values, and include the confidence intervals around your forecast. 

Refer to the guide [here](https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_forecasting.html) for an example of how to use this forecasting package. 
As it's kind of convoluted, most of this code is provided; let me know if you have any difficulties using it. 

In [ ]:
arma = <ADD CODE FOR YOUR PREFERRED ARMA MODEL HERE>
fig, ax = plt.subplots(figsize=(15, 5))

# Construct the forecasts
fcast = arma.get_forecast(len(test_price)).summary_frame()

arma_predictions = arma.predict()
ax.plot(date, stationary_price, label='original data')
predicted_values = arma_predictions[1:].reshape(-1,1)
ax.plot(train_date[1:], predicted_values, 'r', label='fitted line')
forecast_means = fcast['mean'].values.reshape(-1,1)
ax.plot(test_date, forecast_means, 'k--', label='mean forecast')
ax.fill_between(test_date.flatten(), fcast['mean_ci_lower'], fcast['mean_ci_upper'], color='k', alpha=0.1);
plt.legend();

## Collapse to Monthly Values
Here, we repeat the exercises above, but start with a dataset that's collapsed to monthly values. 

In [ ]:
collapsed = msft.groupby(pd.Grouper(freq ='M')).mean()
month_date = collapsed.reset_index().Date.dt.date.values.reshape(-1,1)
month_price = collapsed.High.values.reshape(-1,1)
month_log_price = np.log(month_price)
plt.plot(month_date, month_price)
plt.title("MSFT stock price over time")
plt.show()
plt.plot(month_date, month_log_price)
plt.title("log of MSFT stock price over time")
plt.show()

In [ ]:
<TODO: TRANSFORM TO STATIONARY DATA>
month_stationary = <YOUR TRANSFORMED TIME SERIES>

In [ ]:
month_train_test = int(0.8 * len(month_date))
month_train, month_test = month_stationary[:month_train_test], month_stationary[month_train_test:]
month_date_train, month_date_test = month_date[:month_train_test], month_date[month_train_test:]

In [ ]:
sm.api.graphics.tsa.plot_acf(month_train, lags=14)
plt.show()
sm.api.graphics.tsa.plot_pacf(month_test, lags=14)
plt.show()

In [ ]:
import statsmodels as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
# TODO: determine the order of the ARMA model. 
arma = SARIMAX(month_train, order=<ADD ORDER HERE>).fit()
print(arma.summary())
arma_predictions = arma.predict()
arma_residuals = month_train - arma_predictions.reshape(-1,1)
arma_residuals = arma_residuals[1:] # Fitting AR 1 model means removing one observation
plt.plot(month_date_train, month_train, label='original data')
plt.plot(month_date_train[1:], arma_predictions[1:], 'r', label='fitted line')
plt.show()
plt.plot(month_date_train[1:], arma_residuals, 'o')
plt.show()
print("MSE with selected model:", np.mean(arma_residuals**2))
print("AIC with selected model:", evaluate_AIC(<ADD DOF HERE>, arma_residuals))
print("BIC with selected model:", evaluate_BIC(<ADD DOF HERE>, arma_residuals))

In [ ]:
arma = SARIMAX(month_train, order=<ADD ORDER HERE>).fit()
fig, ax = plt.subplots(figsize=(15, 5))

# Construct the forecasts
fcast = arma.get_forecast(len(month_test)).summary_frame()

arma_predictions = arma.predict()
ax.plot(month_date, month_stationary, label='original data')
predicted_values = arma_predictions[1:].reshape(-1,1)
ax.plot(month_date_train[1:], predicted_values, 'r', label='fitted line')
forecast_means = fcast['mean'].values.reshape(-1,1)
ax.plot(month_date_test, forecast_means, 'k--', label='mean forecast')
ax.fill_between(month_date_test.flatten(), fcast['mean_ci_lower'], fcast['mean_ci_upper'], color='k', alpha=0.1);
plt.legend();